In [1]:
import warnings
import requests
warnings.filterwarnings('ignore')


from pathlib import Path
import os, sys

parent = os.path.abspath('..')
sys.path.insert(1, parent)
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd


from numpy.random import choice, normal

import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from matplotlib.ticker import FuncFormatter


from numba import jit


from matplotlib import cm
from itertools import product

import time
from datetime import datetime, timedelta
import datetime
import plotly.io as pio
import plotly.express as px

import talib

#import gtts
#from playsound import playsound


from scipy.stats import spearmanr
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, explained_variance_score, roc_curve, make_scorer, roc_auc_score, mean_absolute_error, recall_score, precision_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge,  LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split, learning_curve, cross_val_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz, _tree
from sklearn import tree
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestClassifier, RandomForestRegressor,  GradientBoostingClassifier, GradientBoostingRegressor, VotingClassifier




#/Users/johnh/Macro3/Macro
#sys.path.append('/Users/johnh/Macro3/Macro')
pd.set_option('display.expand_frame_repr', False)

sns.set_style('whitegrid')
idx = pd.IndexSlice

pd.options.display.max_rows = 500


from joblib import load




from joblib import Parallel, delayed




In [1]:
import DataHacker as dh

In [2]:
og = dh.Organizer(ticker='amzn')

In [4]:
og.get_pred()

array([array([0]), 56.51], dtype=object)

In [5]:
og.new_data

,RET 1,RET 3,RET 5,RET 8,RET 21,VOL,target
Date,,,,,,,
1999-03-12,NaN,NaN,NaN,NaN,NaN,NaN,1
1999-03-15,0.009740,NaN,NaN,NaN,NaN,-0.436166,1
1999-03-16,0.025723,NaN,NaN,NaN,NaN,0.990260,1
1999-03-17,0.004702,0.040584,NaN,NaN,NaN,-0.080404,1
1999-03-18,0.001560,0.032154,NaN,NaN,NaN,-0.260340,0
...,...,...,...,...,...,...,...
2023-04-11,-0.001528,-0.003921,0.002301,0.012454,0.063303,0.489370,0
2023-04-12,-0.001968,-0.009764,0.003904,0.009567,0.050947,0.009507,1
2023-04-13,0.009969,0.006441,0.004030,0.011410,0.062525,0.188558,0


In [23]:
prices = dh.APIPull('AAPL').data.copy()

In [24]:
data = prices.copy()

In [25]:
list1 = [1,3, 5, 8, 21]
list2 = [1]

for i in list1:
    data[f'RET {i}'] = data['close'].pct_change(i)

data['VOL'] = data['volume'].pct_change(1)

data['target'] = data['RET 1'].shift(-1)
data['target'] = (data['target'] > 0).astype(int)


data = data.drop(['open', 'high', 'low', 'close', 'volume'],axis=1)
data = data.dropna()

In [26]:
data

,RET 1,RET 3,RET 5,target
Date,,,,
1999-03-19,-0.056338,-0.056338,0.009416,1
1999-03-22,0.046642,0.029359,0.029359,0
1999-03-23,-0.058823,-0.070421,-0.070421,1
1999-03-24,0.020832,0.005597,-0.011008,1
1999-03-25,0.003710,-0.035652,-0.047535,0
...,...,...,...,...
2023-04-11,-0.007591,-0.018075,-0.032316,0
2023-04-12,-0.004353,-0.027693,-0.033388,1
2023-04-13,0.034104,0.021786,0.010992,0


In [34]:
def split(input_data):
    data = input_data.copy()
    oos = data['2023':]
    data = data[:'2023']    

    return data, oos

def create_xy(input_data):
    data = input_data.copy()
    y = data.filter(like='target')
    X = data.drop(y.columns, axis=1)

    return X,y



In [35]:
data, oos = split(data)
X, y = create_xy(data)
X2, y2 = create_xy(oos)

In [37]:
estimators_int = 250

max_depth_int = 25

min_split = 90
# min_split = 60

min_leaf = 40

f_leaf = 0.0

features = 'sqrt'


rf_clf = RandomForestClassifier(n_estimators=estimators_int,             # default changed from 10 to 100 in version 0.22 
                                    criterion='entropy', 
                                    max_depth=max_depth_int, 
                                    min_samples_split=min_split, 
                                    min_samples_leaf=min_leaf, 
                                    min_weight_fraction_leaf=f_leaf, 
                                    max_features=features,
                                    max_leaf_nodes=None, 
                                    min_impurity_decrease=0.0, 
                                    bootstrap=True, 
                                    oob_score=True, 
                                    random_state=5,
                                    n_jobs=-1,
                                    warm_start=False, 
                                    verbose=0)
model = rf_clf

In [38]:
model.fit(X, y)

RandomForestClassifier(criterion='entropy', max_depth=25, min_samples_leaf=40,
                       min_samples_split=90, n_estimators=250, n_jobs=-1,
                       oob_score=True, random_state=5)

In [39]:
model.predict(X2)

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0])

In [40]:
joblib.dump(model, 'aapl.joblib')

['aapl.joblib']

In [55]:
list1 = ['AAPL', 'MSFT', 'AMZN', 'GOOG', 'GOOGL', 'NVDA', 'TSLA', 'META', 'PEP', 'AVGO']

In [46]:
prices = dh.APIPull('MSFT').data.copy()

In [51]:
prices = dh.APIPull('TSLA').data.copy()

In [56]:
f'{list1[2]}'

'AMZN'

In [57]:
prices = dh.APIPull(f'{list1[2]}').data.copy()

In [58]:
for i in list1:    
    prices = dh.APIPull(f'{i}').data.copy()

    data = prices.copy()

    list1 = [1,3, 5, 8, 21]
    list2 = [1]

    for n in list1:
        data[f'RET {n}'] = data['close'].pct_change(n)

    data['VOL'] = data['volume'].pct_change(1)

    data['target'] = data['RET 1'].shift(-1)
    data['target'] = (data['target'] > 0).astype(int)


    data = data.drop(['open', 'high', 'low', 'close', 'volume'],axis=1)
    data = data.dropna()





    data, oos = split(data)
    X, y = create_xy(data)
    X2, y2 = create_xy(oos)

    estimators_int = 250

    max_depth_int = 25

    min_split = 90
    # min_split = 60

    min_leaf = 40

    f_leaf = 0.0

    features = 'sqrt'


    rf_clf = RandomForestClassifier(n_estimators=estimators_int,             # default changed from 10 to 100 in version 0.22 
                                        criterion='entropy', 
                                        max_depth=max_depth_int, 
                                        min_samples_split=min_split, 
                                        min_samples_leaf=min_leaf, 
                                        min_weight_fraction_leaf=f_leaf, 
                                        max_features=features,
                                        max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0, 
                                        bootstrap=True, 
                                        oob_score=True, 
                                        random_state=5,
                                        n_jobs=-1,
                                        warm_start=False, 
                                        verbose=0)
    model = rf_clf
    model.fit(X, y)
    joblib.dump(model, '{}.joblib'.format(i.lower()))


In [43]:
for i in list1:
    print(i.lower())

aapl
msft
amzn
goog
googl
nvda
tsla
meta
pep
avgo
